In [3]:
import requests # library to handle requests
import pandas as pd # library for data analysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
CLIENT_ID = 'U0YRC25UFQQXCML51BQIAVOHH3GIR4J2LSISOSPY4PRP4U1J' # your Foursquare ID
CLIENT_SECRET = 'S2CKFPHFDXCVWO4UI02A20OMLSRF2VRJI02G53BALGXZVI2A' # your Foursquare Secret
ACCESS_TOKEN = 'fsq3EWjrDVS7MmpNxPS2oYo9CdBphNOQ/lOuCT1N7N4xCzo=' # your FourSquare Access Token
VERSION = '20221205'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [10]:
address = 'Koningin Astridplein 27, 2018 Antwerpen, Belgium'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

51.2162206 4.421068612450207


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [36]:
import requests

url = "https://api.foursquare.com/v3/places/search"

params = {
  "query": "restaurants",
    #"open_now": "true",
  "ll": "51.2162206,4.421068612450207",
  "sort":"rating"
}

headers = {
    "Accept": "application/json",
    "Authorization": "fsq3EWjrDVS7MmpNxPS2oYo9CdBphNOQ/lOuCT1N7N4xCzo="
}

response = requests.request("GET", url, params=params, headers=headers)

print(response.text)
response

{"results":[{"fsq_id":"4b72fa53f964a52087932de3","categories":[{"id":13064,"name":"Pizzeria","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/pizza_","suffix":".png"}},{"id":13236,"name":"Italian Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/italian_","suffix":".png"}}],"chains":[],"distance":2233,"geocodes":{"main":{"latitude":51.197263,"longitude":4.410711},"roof":{"latitude":51.197263,"longitude":4.410711}},"link":"/v3/places/4b72fa53f964a52087932de3","location":{"address":"Generaal Lemanstraat 2A","country":"BE","cross_street":"","formatted_address":"Generaal Lemanstraat 2A, 2600 Antwerpen","locality":"Antwerp","neighborhood":["Stad"],"postcode":"2600","region":"Antwerp"},"name":"Il Carpaccio","related_places":{},"timezone":"Europe/Brussels"},{"fsq_id":"4dfd0303227185f38b96eeb8","categories":[{"id":13026,"name":"BBQ Joint","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/bbqalt_","suffix":".png"}}],"chains":[],"distance":8145,"g

<Response [200]>

Put your parsed results into a DataFrame

In [85]:
res_json = response.json()
print(res_json.keys())

dict_keys(['results', 'context'])


In [86]:
res_json['results']

[{'fsq_id': '4b72fa53f964a52087932de3',
  'categories': [{'id': 13064,
    'name': 'Pizzeria',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
     'suffix': '.png'}},
   {'id': 13236,
    'name': 'Italian Restaurant',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
     'suffix': '.png'}}],
  'chains': [],
  'distance': 2233,
  'geocodes': {'main': {'latitude': 51.197263, 'longitude': 4.410711},
   'roof': {'latitude': 51.197263, 'longitude': 4.410711}},
  'link': '/v3/places/4b72fa53f964a52087932de3',
  'location': {'address': 'Generaal Lemanstraat 2A',
   'country': 'BE',
   'cross_street': '',
   'formatted_address': 'Generaal Lemanstraat 2A, 2600 Antwerpen',
   'locality': 'Antwerp',
   'neighborhood': ['Stad'],
   'postcode': '2600',
   'region': 'Antwerp'},
  'name': 'Il Carpaccio',
  'related_places': {},
  'timezone': 'Europe/Brussels'},
 {'fsq_id': '4dfd0303227185f38b96eeb8',
  'categories': [{'id': 13026,
    'name'

In [87]:
fs_df = pd.json_normalize(res_json['results'])
fs_df.head(2)

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,geocodes.roof.longitude,location.address,location.country,location.cross_street,location.formatted_address,location.locality,location.neighborhood,location.postcode,location.region,related_places.children
0,4b72fa53f964a52087932de3,"[{'id': 13064, 'name': 'Pizzeria', 'icon': {'p...",[],2233,/v3/places/4b72fa53f964a52087932de3,Il Carpaccio,Europe/Brussels,51.197263,4.410711,51.197263,4.410711,Generaal Lemanstraat 2A,BE,,"Generaal Lemanstraat 2A, 2600 Antwerpen",Antwerp,[Stad],2600,Antwerp,NaN
1,4dfd0303227185f38b96eeb8,"[{'id': 13026, 'name': 'BBQ Joint', 'icon': {'...",[],8145,/v3/places/4dfd0303227185f38b96eeb8,'t Hoeveke,Europe/Brussels,51.171796,4.514100,NaN,NaN,Hoogveldweg 135,BE,,"Hoogveldweg 135, 2500 Lier",Lier,NaN,2500,Antwerp,NaN


In [88]:
print(fs_df.shape)

(10, 20)


In [89]:
cat_df = pd.json_normalize(fs_df['categories'].iloc[0])
cat_df.head(5)

,id,name,icon.prefix,icon.suffix
0,13064,Pizzeria,https://ss3.4sqi.net/img/categories_v2/food/pi...,.png
1,13236,Italian Restaurant,https://ss3.4sqi.net/img/categories_v2/food/it...,.png


In [95]:
name_df = fs_df[['name']]
name_df.head(20)

,name
0,Il Carpaccio
1,'t Hoeveke
2,Hofke van Bazel
3,Pazzo
4,The Jane
5,Centpourcent
6,Het Gebaar
7,Bia Mara
8,Frituur Lescluze
9,Kartini


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [23]:
import requests
import json
import pandas as pd
import json
import requests
import scipy as sp
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [49]:
api_key='HQroL6ytG5N3erZTbXQLhbzFp8O7__8QRU0pjSs95CYb4DcaryRxbXgfNaU2pi2LdPnT4J6puxatRShMU2SSAobf2PbLzSGcq_KmUadaFecOppG6gQcWqbfrqT2OY3Yx'
headers = {'Authorization': 'Bearer %s' % api_key}

In [54]:
url='https://api.yelp.com/v3/businesses/search'
 
# In the dictionary, term can take values like food, cafes or businesses like McDonalds
params = {'term':'restaurants','location':'Koningin Astridplein 27, 2018 Antwerpen, Belgium'}

In [55]:
# Making a get request to the API
req=requests.get(url, params=params, headers=headers)
 
# proceed only if the status code is 200
print('The status code is {}'.format(req.status_code))

The status code is 200


In [56]:
# printing the text from the response 
json.loads(req.text)
data = json.loads(req.text)

In [59]:
data["businesses"][1]["name"]

"Bistrorant 't Hofke"

In [60]:
len(data)

3

In [61]:
data.keys()

dict_keys(['businesses', 'total', 'region'])

In [96]:
data['businesses']

[{'id': 'CzH-VL1Trel7mrofnX75BQ',
  'alias': 'de-arme-duivel-antwerpen',
  'name': 'De Arme Duivel',
  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/zZo3Ntni1h8DbK8Rp3eTUQ/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/de-arme-duivel-antwerpen?adjust_creative=LKMCTcauyJPAlbY-7QDYzA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=LKMCTcauyJPAlbY-7QDYzA',
  'review_count': 50,
  'categories': [{'alias': 'belgian', 'title': 'Belgian'}],
  'rating': 4.5,
  'coordinates': {'latitude': 51.21641, 'longitude': 4.40804},
  'transactions': [],
  'price': '€€',
  'location': {'address1': 'Armeduivelstraat 1',
   'address2': None,
   'address3': None,
   'city': 'Antwerp',
   'zip_code': '2000',
   'country': 'BE',
   'state': 'VAN',
   'display_address': ['Armeduivelstraat 1', '2000 Antwerp', 'Belgium']},
  'phone': '+3232322698',
  'display_phone': '+32 3 232 26 98',
  'distance': 877.8035957144624},
 {'id': 'CkXsPkwg32P6Kp4laRXN-A',
  'alias': 'bi

In [97]:
yelp_df = pd.json_normalize(data['businesses'])
yelp_df.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,CzH-VL1Trel7mrofnX75BQ,de-arme-duivel-antwerpen,De Arme Duivel,https://s3-media1.fl.yelpcdn.com/bphoto/zZo3Nt...,False,https://www.yelp.com/biz/de-arme-duivel-antwer...,50,"[{'alias': 'belgian', 'title': 'Belgian'}]",4.5,[],...,51.21641,4.40804,Armeduivelstraat 1,None,None,Antwerp,2000,BE,VAN,"[Armeduivelstraat 1, 2000 Antwerp, Belgium]"
1,CkXsPkwg32P6Kp4laRXN-A,bistrorant-t-hofke-antwerpen,Bistrorant 't Hofke,https://s3-media3.fl.yelpcdn.com/bphoto/FCKR06...,False,https://www.yelp.com/biz/bistrorant-t-hofke-an...,28,"[{'alias': 'belgian', 'title': 'Belgian'}]",4.5,[],...,51.22001,4.39941,Oude Koornmarkt 16,None,None,Antwerp,2000,BE,VAN,"[Oude Koornmarkt 16, 2000 Antwerp, Belgium]"


In [98]:
print(yelp_df.shape)

(20, 24)


In [107]:
name2_df = yelp_df[['name', 'rating']]
name2_df.head(20)

,name,rating
0,De Arme Duivel,4.5
1,Bistrorant 't Hofke,4.5
2,Chez Albert,4.5
3,Queen A's Kitchen,4.5
4,Roma,4.0
5,Hans Christian Andersen,4.0
6,Ni Shifu,4.5
7,De Pottekijker,4.5
8,Eighteen,5.0
9,Boston Steak House,3.5


Put your parsed results into a DataFrame

# Comparing Results

Conclusion: Yelp Provided more details, 10 more places than Foursquare. Working with Yelp was much easier and took me less time. If I have to ever chose one,I would go with Yelp API

Get the top 20 restaurants according to their rating

In [108]:
name2_df.head(20)

,name,rating
0,De Arme Duivel,4.5
1,Bistrorant 't Hofke,4.5
2,Chez Albert,4.5
3,Queen A's Kitchen,4.5
4,Roma,4.0
5,Hans Christian Andersen,4.0
6,Ni Shifu,4.5
7,De Pottekijker,4.5
8,Eighteen,5.0
9,Boston Steak House,3.5
